#  Data Cleaning and Preprocessing of Movie Dataset

This process cleans and preprocesses a movie dataset by:

* Removing duplicate rows.
* Handling missing 'overview' values.
* Cleaning text in the 'overview' column (removing special characters, lemmatizing, and removing stopwords).
* Filtering out movies with unrealistic release years.
* Ensuring proper data types for columns.
* Checking for unusual values in the 'language' column.
* Saving the cleaned dataset for further analysis.

In [14]:
# Import necessary libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
# Load cleaned dataset
df_cleaned = pd.read_csv('data/movies_2015_2023_genres.csv')
df_cleaned.head()

,title,release_year,language,genre,overview,vote_average,vote_count,popularity
0,Pad Man,2018.0,Indisch,Humor,Upon realizing the extent to which women are a...,7.420,200.0,7.036
1,Tamasha,2015.0,Indisch,Humor,"Meeting while on vacation, Ved and Tara sense ...",6.720,141.0,8.770
2,Tu Jhoothi Main Makkaar,2023.0,Indisch,Humor,"To earn extra cash, Mickey helps couples break...",6.253,144.0,10.045
3,Hindi Medium,2017.0,Indisch,Humor,"Mita and Raj Batra, an affluent couple from De...",7.300,166.0,7.001
4,Dilwale,2015.0,Indisch,Humor,Raj is a Mafia member. One day he meet a girl ...,6.648,301.0,11.501


In [16]:
# Remove .0 from release_year
df_cleaned['release_year'] = df_cleaned['release_year'].astype(int)
df_cleaned.head()

,title,release_year,language,genre,overview,vote_average,vote_count,popularity
0,Pad Man,2018,Indisch,Humor,Upon realizing the extent to which women are a...,7.420,200.0,7.036
1,Tamasha,2015,Indisch,Humor,"Meeting while on vacation, Ved and Tara sense ...",6.720,141.0,8.770
2,Tu Jhoothi Main Makkaar,2023,Indisch,Humor,"To earn extra cash, Mickey helps couples break...",6.253,144.0,10.045
3,Hindi Medium,2017,Indisch,Humor,"Mita and Raj Batra, an affluent couple from De...",7.300,166.0,7.001
4,Dilwale,2015,Indisch,Humor,Raj is a Mafia member. One day he meet a girl ...,6.648,301.0,11.501


In [17]:
# Step 1: Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Step 2: Handle missing values
# Drop rows with missing 'overview' since it's crucial for the task
df_cleaned = df_cleaned.dropna(subset=['overview'])
print(df_cleaned.isna().sum())


title           0
release_year    0
language        0
genre           0
overview        0
vote_average    2
vote_count      2
popularity      2
dtype: int64


In [18]:
# Step 3: Text Cleaning - Clean 'overview' text
stop_words = set(stopwords.words('english')).union(set(stopwords.words('german')))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower().strip()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words)
    return text

df_cleaned['overview'] = df_cleaned['overview'].apply(clean_text)

In [19]:

# Step 4: Outlier Detection and Removal for 'release_year'
# Remove rows with unrealistic release years (e.g., below 1900 or beyond 2023)
df_cleaned = df_cleaned[(df_cleaned['release_year'] >= 1900) & (df_cleaned['release_year'] <= 2023)]

# Step 5: Convert data types - Ensure 'release_year' is an integer
df_cleaned['release_year'] = df_cleaned['release_year'].astype(int)

# Step 6: Check for unusual values in 'language' column
unusual_languages = df_cleaned[~df_cleaned['language'].isin(['Amerikanisch', 'Indisch', 'Französisch', 'Spanisch', 'Deutsch', 'Italienisch'])]
print(f"unusual values in 'language':\n{unusual_languages}")

unusual values in 'language':
Empty DataFrame
Columns: [title, release_year, language, genre, overview, vote_average, vote_count, popularity]
Index: []


In [20]:
# Step 8: Updated frequency count
language_counts = df_cleaned['language'].value_counts()
print(f"Number of films per language after adjustment:\n{language_counts}")

Number of films per language after adjustment:
language
Amerikanisch    4480
Spanisch        3197
Französisch     2519
Deutsch         1911
Indisch         1873
Italienisch     1520
Name: count, dtype: int64


In [21]:
# Save cleaned data to CSV
df_cleaned.to_csv('data/cleaned_movies_2015_2023_genres.csv', index=False)
print(f"Total number of movies after cleaning: {len(df_cleaned)}")

Total number of movies after cleaning: 15500
